Isotope Correction Notebook

In [1]:
#Import all the necessary libraries
import pymzml
import csv
import os
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
import re
import plotly.express as px
from IPython.display import Image

No module named 'ms_deisotope._c.averagine' averagine
No module named 'ms_deisotope._c.scoring'
No module named 'ms_deisotope._c.deconvoluter_base'
No module named 'ms_deisotope._c.deconvoluter_base'
No module named 'ms_deisotope._c.deconvoluter_base'


In [64]:
test = pd.read_csv('data_OzESI/isotope/isotope_csv/RT 13.2_NL271.csv', skiprows=1)

#change column X(Thomsons)) to m/z
test['m/z'] = test.loc[:,'X(Thomsons)']
test['intensity'] = test.loc[:,'Y(Counts)']
test = test.drop(['X(Thomsons)'], axis=1)
test = test.drop(['Y(Counts)'], axis=1)
test = test.drop(['#Point'], axis=1)
test.head()

,m/z,intensity
0,700.0,41.899998
1,700.1,2612.440186
2,700.2,3349.796143
3,700.3,3049.846191
4,700.4,1930.580078


In [66]:
#find 874 in the m/z column
#test.loc[test['m/z'] == 874.8]

def find_mz_row(mz):
    print(test.loc[test['m/z'] == mz])

find_mz_row(874.8)

        m/z     intensity
1748  874.8  18602.994141


In [67]:
#create a function that inputs the m/z value and intensity and outputs 3 m/z values and 3 intensities at 60% of the original intensity, 19.5% of the original intensity and 0.5% of the original intensity and 4% of the original intensity
def isotope_correction(mz, intensity, dataframe):
    mz1 = mz
    mz2 = mz + 1
    mz3 = mz + 2
    mz4 = mz + 3
    intensity1 = intensity
    intensity2 = intensity * 0.609
    intensity3 = intensity * 0.195
    intensity4 = intensity * 0.042
    isotope1 = 100
    isotope2 = 60.9
    isotope3 = 19.5
    isotope4 = 4.2
    print('mz1: ',mz1,'intensity1: ', float(intensity1.round(0)),  '\nmz2: ', 
          mz2,'intensity2: ', float(intensity2.round(0)), '\nmz3: ', 
          mz3,'intensity3: ', float(intensity3.round(0)), '\nmz4: ', 
          mz4,    'intensity4: ', float(intensity4.round(0))
    )
    #append all the new values to the dataframe
    dataframe.loc[len(dataframe)] = [mz1, intensity1.round(0), isotope1]
    dataframe.loc[len(dataframe)] = [mz2, intensity2.round(0), isotope2]
    dataframe.loc[len(dataframe)] = [mz3, intensity3.round(0), isotope3]
    dataframe.loc[len(dataframe)] = [mz4, intensity4.round(0), isotope4]
    return mz1, mz2, mz3, mz4, intensity1, intensity2, intensity3, intensity4

In [69]:
df = pd.DataFrame(columns=['m/z', 'intensity', 'Isotope %'])
isotope_correction(test.iloc[1748,0], test.iloc[1748,1], df)

mz1:  874.8 intensity1:  18603.0 
mz2:  875.8 intensity2:  11329.0 
mz3:  876.8 intensity3:  3628.0 
mz4:  877.8 intensity4:  781.0


(874.8,
 875.8,
 876.8,
 877.8,
 18602.994140625,
 11329.223431640625,
 3627.583857421875,
 781.32575390625)

In [70]:
df

,m/z,intensity,Isotope %
0,874.8,18603.0,100.0
1,875.8,11329.0,60.9
2,876.8,3628.0,19.5
3,877.8,781.0,4.2


In [71]:
# Zm = [(Nc * 1.12) * (Nc * 1.12)/200 + No * 0.204)]/100 
#Nc number of carbons
#No number of oxygens

def Zm(Nc,No):
    Zm = ((Nc * 1.12) * (Nc * 1.12)/200 + No * 0.204)/100
    return Zm

In [72]:
Zm(57,6)

0.21601728